In [1]:
import json

def parse_run_results(run_dict: dict):
    runs_to_parsed_results = {}
    for name, json_path in run_dict.items():
        runs_to_parsed_results[name] = {}
        timesteps = []
        episodes = []
        exploitability = []
        print(f"parsing {json_path}")
        with open(json_path, "r") as json_file:
            for line in json_file:
                try:
                    json_result = json.loads(s=line)
                except json.JSONDecodeError:
                    break

                timesteps_entry = json_result["timesteps_total"]
                episodes_entry = json_result["episodes_total"]
                try:
                    exploitability_entry = (json_result.get("avg_policy_exploitability") or
                                            json_result.get("z_avg_policy_exploitability") or
                                            json_result.get("exploitability") or
                                            json_result.get("approx_exploitability"))
                    if exploitability_entry is None:
                        raise KeyError
                    if not any(tag in json_path for tag in ["openspiel", "sparse", "xfdo", "nxdo", "no_limit"]):
                        for i in range(99):
                            try:
                                next(json_file)
                            except StopIteration:
                                break
                            except UnicodeDecodeError:
                                continue
                except KeyError:
                    continue

                timesteps.append(timesteps_entry)
                episodes.append(episodes_entry)
                exploitability.append(exploitability_entry)

        runs_to_parsed_results[name]["timesteps"] = timesteps
        runs_to_parsed_results[name]["episodes"] = episodes
        runs_to_parsed_results[name]["exploitability"] = exploitability
    return runs_to_parsed_results


In [2]:
import os
import tarfile
from grl.utils.common import data_dir

# extract m_clone_poker_data.tar.gz, containing data to graph, if we haven't already done so
if not os.path.exists(f"{data_dir()}/20x_dummy_leduc_nfsp_dqn_gpu_sparse_08.29.31PM_Jan-27-2021zmalq5uj/result.json"):
    poker_data_path = f"{data_dir()}/m_clone_poker_data.tar.gz"
    print(f"extracting {poker_data_path}")
    tar = tarfile.open(poker_data_path, "r:gz")
    tar.extractall(path=data_dir())
    tar.close()


extracting /home/jb/git/grl/grl/data/m_clone_poker_data.tar.gz


In [3]:
# Add your own data here. Note that PSRO runs are parsed separately.
nxdo_and_nfsp_runs = {
    "20-Clone Leduc NFSP seed 1": f"{data_dir()}/20x_dummy_leduc_nfsp_dqn_gpu_sparse_08.29.31PM_Jan-27-2021zmalq5uj/result.json",
    "20-Clone Leduc NFSP seed 2": f"{data_dir()}/20x_dummy_leduc_nfsp_dqn_gpu_sparse_08.29.32PM_Jan-27-2021448z517h/result.json",
    "20-Clone Leduc NFSP seed 3": f"{data_dir()}/20x_dummy_leduc_nfsp_dqn_gpu_sparse_08.29.34PM_Jan-27-20219a6jc3bj/result.json",

    "20-Clone Leduc NXDO (Ours) seed 1": f"{data_dir()}/20x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_08.20.33PM_Jan-27-2021/manager_results.json",
    "20-Clone Leduc NXDO (Ours) seed 2": f"{data_dir()}/20x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_08.29.02PM_Jan-27-2021/manager_results.json",
    "20-Clone Leduc NXDO (Ours) seed 3": f"{data_dir()}/20x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_08.29.06PM_Jan-27-2021/manager_results.json",

    "20-Clone Leduc NXDO-VA (Ours) seed 1": f"{data_dir()}/va_20x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_08.29.10PM_Jan-27-2021/manager_results.json",
    "20-Clone Leduc NXDO-VA (Ours) seed 2": f"{data_dir()}/va_20x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_08.29.12PM_Jan-27-2021/manager_results.json",
    "20-Clone Leduc NXDO-VA (Ours) seed 3": f"{data_dir()}/va_20x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_08.29.14PM_Jan-27-2021/manager_results.json",

    "40-Clone Leduc NFSP seed 1": f"{data_dir()}/40x_dummy_leduc_nfsp_dqn_gpu_sparse_10.17.16PM_Jan-29-2021p9347we9/result.json",
    "40-Clone Leduc NFSP seed 2": f"{data_dir()}/40x_dummy_leduc_nfsp_dqn_gpu_sparse_03.22.12AM_Feb-01-2021x3doc48r/result.json",
    "40-Clone Leduc NFSP seed 3": f"{data_dir()}/40x_dummy_leduc_nfsp_dqn_gpu_sparse_03.22.21AM_Feb-01-2021a2pbfxcy/result.json",

    "40-Clone Leduc NXDO-VA (Ours) seed 1": f"{data_dir()}/40x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_10.32.40PM_Jan-29-2021/manager_results.json",
    "40-Clone Leduc NXDO-VA (Ours) seed 2": f"{data_dir()}/va_40x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_10.30.57PM_Feb-02-2021/manager_results.json",
    "40-Clone Leduc NXDO-VA (Ours) seed 3": f"{data_dir()}/va_40x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_10.31.48PM_Feb-02-2021/manager_results.json",

    "80-Clone Leduc NFSP seed 1": f"{data_dir()}/80x_dummy_leduc_nfsp_dqn_gpu_sparse_10.16.53PM_Jan-29-2021wa_a_w0w/result.json",
    "80-Clone Leduc NFSP seed 2": f"{data_dir()}/80x_dummy_leduc_nfsp_dqn_gpu_sparse_03.10.09AM_Feb-01-2021k8g2kjub/result.json",
    "80-Clone Leduc NFSP seed 3": f"{data_dir()}/80x_dummy_leduc_nfsp_dqn_gpu_sparse_03.10.17AM_Feb-01-2021nm0cc7zc/result.json",

    "80-Clone Leduc NXDO-VA (Ours) seed 1": f"{data_dir()}/va_80x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_09.23.05PM_Jan-31-2021/manager_results.json",
    "80-Clone Leduc NXDO-VA (Ours) seed 2": f"{data_dir()}/va_80x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_03.15.40AM_Feb-01-2021/manager_results.json",
    "80-Clone Leduc NXDO-VA (Ours) seed 3": f"{data_dir()}/va_80x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_03.16.57AM_Feb-01-2021/manager_results.json",
}

psro_runs = {
    "20-Clone Leduc DQN-PSRO seed 1": f"{data_dir()}/seed_0_20x_dummy_leduc_psro_dqn_gpu_08.29.17PM_Jan-27-2021.json",
    "20-Clone Leduc DQN-PSRO seed 2": f"{data_dir()}/seed_1_20x_dummy_leduc_psro_dqn_gpu_08.29.19PM_Jan-27-2021.json",
    "20-Clone Leduc DQN-PSRO seed 3": f"{data_dir()}/seed_2_20x_dummy_leduc_psro_dqn_gpu_08.29.25PM_Jan-27-2021.json",

    "40-Clone Leduc DQN-PSRO seed 1": f"{data_dir()}/40x_dummy_leduc_psro_dqn_gpu_03.33.30AM_Feb-01-2021.json",
    "40-Clone Leduc DQN-PSRO seed 2": f"{data_dir()}/40x_dummy_leduc_psro_dqn_gpu_04.59.02AM_Feb-03-2021.json",
    "40-Clone Leduc DQN-PSRO seed 3": f"{data_dir()}/40x_dummy_leduc_psro_dqn_gpu_02.41.27AM_Feb-04-2021.json",

    "80-Clone Leduc DQN-PSRO seed 1": f"{data_dir()}/80x_dummy_leduc_psro_dqn_gpu_03.13.11AM_Feb-01-2021.json",
    "80-Clone Leduc DQN-PSRO seed 2": f"{data_dir()}/80x_dummy_leduc_psro_dqn_gpu_03.11.23AM_Feb-01-2021.json",
    "80-Clone Leduc DQN-PSRO seed 3": f"{data_dir()}/80x_dummy_leduc_psro_dqn_gpu_09.24.25PM_Jan-31-2021.json",
}


In [4]:
import itertools
import pandas as pd
import plotly.express as px
import plotly.io as pio
from grl.utils.common import datetime_str
# pio.renderers.default = 'png'
pio.renderers.default = "browser"


runs_to_parsed_results = parse_run_results(run_dict=nxdo_and_nfsp_runs)

timesteps = list(itertools.chain(*[v["timesteps"] for k, v in runs_to_parsed_results.items()]))
episodes = list(itertools.chain(*[v["episodes"] for k, v in runs_to_parsed_results.items()]))

exploitability = list(itertools.chain(*[v["exploitability"] for k, v in runs_to_parsed_results.items()]))
run = list(itertools.chain(*[[k] * len(v["exploitability"]) for k, v in runs_to_parsed_results.items()]))

in_dict = {
    "timesteps": timesteps,
    "episodes": episodes,
    "exploitability": exploitability,
    "run": run,
}

for run_name, data_path in psro_runs.items():
    with open(data_path, "r") as json_file:
        data = json.load(json_file)

    filtered_data = {}
    for key, column in data.items():
        filtered_data[key] = []
        for i, item in enumerate(column):
            if data["exploitability"][i] is not None:
                filtered_data[key].append(data[key][i])
    data = filtered_data

    timesteps = data.get("timesteps") or data.get("timesteps_total") or data["total_steps"]
    episodes = data.get("episodes") or data.get("episodes_total") or data["total_episodes"]

    assert len(timesteps) == len(episodes)
    assert len(data["exploitability"]) == len(timesteps)

    in_dict["timesteps"].extend(timesteps)
    in_dict["episodes"].extend(episodes)
    in_dict["exploitability"].extend(data["exploitability"])
    in_dict["run"].extend(run_name for _ in data["exploitability"])


df = pd.DataFrame.from_dict(data=in_dict)
fig = px.line(df, x="episodes", y="exploitability", color="run", title='Poker Exploitability')

# toggles runs to be invisible by default
fig.for_each_trace(lambda trace: trace.update(visible="legendonly"))

fig.add_annotation(
                text="Double-click the legend to toggle visibility of all runs.",
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                xanchor="left",
                x=1,
                y=0.0,
                bordercolor=None,
                borderwidth=1
)

fig.show()
fig.write_html(f"/tmp/exploitability_{datetime_str()}.html")

parsing /home/jb/git/grl/grl/data/20x_dummy_leduc_nfsp_dqn_gpu_sparse_08.29.31PM_Jan-27-2021zmalq5uj/result.json
parsing /home/jb/git/grl/grl/data/20x_dummy_leduc_nfsp_dqn_gpu_sparse_08.29.32PM_Jan-27-2021448z517h/result.json
parsing /home/jb/git/grl/grl/data/20x_dummy_leduc_nfsp_dqn_gpu_sparse_08.29.34PM_Jan-27-20219a6jc3bj/result.json
parsing /home/jb/git/grl/grl/data/20x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_08.20.33PM_Jan-27-2021/manager_results.json
parsing /home/jb/git/grl/grl/data/20x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_08.29.02PM_Jan-27-2021/manager_results.json
parsing /home/jb/git/grl/grl/data/20x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_08.29.06PM_Jan-27-2021/manager_results.json
parsing /home/jb/git/grl/grl/data/va_20x_dummy_leduc_xfdo_dqn_nfsp_gpu_dynamic_threshold_1_aggressive/manager_08.29.10PM_Jan-27-2021/manager_results.json
parsing /home/jb/git/grl/grl/data/va_20x_dummy_leduc_x